# New notebook to keep dev clean

In [ ]:
import os
import sys
import praw
from praw import Reddit

class WallpaperScraper:
    reddit_oath = {
        'client_id': 'pIxpnAoiGfwE-g',
        'client_secret': 'Ueccpv4dJegXUYbmAIdwxevxjDs',
        'user_agent': 'wallpapers'
    }
    
    imgur_oath = {
        'client_id': '29b27fc1363aa92',
        'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
    }
    
    imgur_headers = {
      'Authorization': f'Client-ID {imgur_oath["client_id"]}'
    }
    
#     sort_options = {'controversial', 'gilded', 'hot', 'new', 'rising', 'top'}
#     time_filter = {'all', 'day', 'hour', 'month', 'week', 'year'}
    
    def __init__(self, img_dir, sort='top', time_filter='week'):
        """
        Initialize a wallpaper subreddit scraper instance with feed options.
        """
        
        self._img_dir = img_dir
        self._mk_img_dir()
        
        self._wallpapers = Reddit(**self.reddit_oath).subreddit('wallpapers')
        if sort in {'controversial', 'top'}:
            self._wallpapers = getattr(self._wallpapers, sort)(limit=1000, time_filter=time_filter)
        else:
            self._wallpapers = getattr(self._wallpapers, sort)(limit=1000)
        
    def _mk_img_dir(self):
        """
        Checks and initializes a directory for image files to be downloaded.
        """
        if not os.path.exists(self._img_dir):
            os.mkdir(self._img_dir)
        if not os.path.exists(os.path.join(self._img_dir, 'buffer')):
            os.mkdir(os.path.join(self._img_dir, 'buffer'))
            
    def _img_url_extract(self, img_sub):
        """
        Extract and return the url hyperlink of only PRAW's submission-type objects.
        """
        if isinstance(img_sub, praw.models.reddit.submission.Submission):
            return img_sub.url
        return None

In [ ]:
test = WallpaperScraper('wallpapers', 'top', 'year')

In [ ]:
for idx, img_sub in enumerate(test._wallpapers):
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        img_sub.url.split('.')[-1]
        file_ext = img_sub.url.split('.')[-1]
        print(img_sub.url, file_ext, img_sub.thumbnail)
    #     with open(f'tmp/tmp{idx}.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.url, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    #     file_ext = img_sub.thumbnail.split('.')[-1]
    #     with open(f'tmp/tmp{idx}_.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.thumbnail, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    
    if idx == 10:
        break

# Need to develop a class - image generator

- Scrape only scrapable images
    - Predictable scrapes: e.g. imgur/reddit
        - Reddit API provides a thumbnail!! - Great for speed-up of image processing and IO
    - Unpredictable scrapes: Other sites/reddit comments
- Develop a pre-buffer
    - Image pops are quick, but buffer can buffer in the background
    - Store image in buffer folder
    > 1 - URL Buffer
    
    > 2 - Image Buffer

In [ ]:
def _img_url_extract(img_sub):
    """
    Extract and return the url hyperlink of only PRAW's submission-type objects.
    Include the link to a thumbnail if it exists
    """
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        return (img_sub.url, img_sub.thumbnail)
    return None

In [ ]:
def is_submission(img_sub):
    return isinstance(img_sub, praw.models.reddit.submission.Submission)

In [ ]:
def is_img(url):
    return url.split('.')[-1] in {'jpg','png'}

In [ ]:
def is_imgur(url):
    return 'imgur' in url

In [ ]:
imgur_oath = {
    'client_id': '29b27fc1363aa92',
    'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
}

imgur_headers = {
    'Authorization': f'Client-ID {imgur_oath["client_id"]}'
}

def get_imgur_img(url):
    img_urls = []
    imgur_hash = url.strip('/').split('/')[-1].split('?')[0]
    
    # Try album endpoint API
    api_url = f'https://api.imgur.com/3/album/{imgur_hash}/images'
    response = requests.request('GET', api_url, headers = imgur_headers, allow_redirects=False)
    if response.ok:
        img_urls += [(image['link'], None) for image in json.loads(response.text)['data']]
    
    else:
        # Try gallery endpoint API
        api_url = f'https://api.imgur.com/3/gallery/album/{gallery_hash}'
        response = requests.request('GET', api_url, headers = imgur_headers, allow_redirects=False)
        if response.ok:
            img_urls += [(image['link'], None) for image in json.loads(response.text)['data']['images']]

    if len(img_urls) == 0:
        print(img_url, 'ERROROROROROROROROROROR')
        
    return img_urls

In [ ]:
get_imgur_img('https://imgur.com/a/WepxW')

In [ ]:
get_imgur_img('https://imgur.com/a/rSZlZ')

In [ ]:
get_imgur_img('https://imgur.com/a/22D4xOJ')

In [ ]:
test = WallpaperScraper('wallpapers', 'top', 'year')

In [ ]:
import requests
import json

urls = []
non_reddit = 0
for idx, img_sub in enumerate(test._wallpapers):
    if is_submission(img_sub):
        if is_img(img_sub.url):
            urls.append((img_sub.url, img_sub.thumbnail))
        elif is_imgur(img_sub.url):
            urls += get_imgur_img(img_sub.url)
        else:
            non_reddit += 1
    
    if idx == 500:
        break

In [ ]:
len(urls)

In [ ]:
non_reddit

In [ ]:
urls

# Putting it togther

In [25]:
import requests
import json
from praw import Reddit
from praw.models.reddit.submission import Submission

class WallpaperScraper:
    """
    Initializes and creates an instance of scraped submissions to the /r/subreddit.
    Stores submissions in the instance's 'wallpapers' attribute.
    """
    
    reddit_oath = {
        'client_id': 'pIxpnAoiGfwE-g',
        'client_secret': 'Ueccpv4dJegXUYbmAIdwxevxjDs',
        'user_agent': 'wallpapers'
    }
    
    imgur_oath = {
        'client_id': '29b27fc1363aa92',
        'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
    }
    
    imgur_headers = {
      'Authorization': f'Client-ID {imgur_oath["client_id"]}'
    }
    
    
    def __init__(self, sort='top', time_filter='week', limit=300):
        """
        Initialize a wallpaper subreddit scraper instance with feed options.
        Stores wallpaper urls in a 'wallpaper' instance attribute'.
        
        Sort options: 'top' (default), 'controversial', 'gilded', 'hot', 'new', 'rising'
        Time filters: 'week' (default), 'all', 'day', 'hour', 'month', 'year'
        Limit: Scrapes up to 1000 (max) submissions, 300 by default
        """
        self.wallpapers = []
        
        with Reddit(**self.reddit_oath) as R:
            r_wallpapers = R.subreddit('wallpapers')
            
            # sort_options = {'controversial', 'gilded', 'hot', 'new', 'rising', 'top'}
            # time_filter = {'all', 'day', 'hour', 'month', 'week', 'year'}
            
            if sort in {'controversial', 'top'}:
                r_wallpapers = getattr(r_wallpapers, sort)(limit=limit, time_filter=time_filter)
            else:
                r_wallpapers = getattr(r_wallpapers, sort)(limit=limit)
                
            for submission in r_wallpapers:
                self._get_wallpaper_url(submission)
                
                
    def __iter__(self):
        return iter(self.wallpapers)

    
    @staticmethod
    def get_imgur(url):
        """
        Queries the Imgur API with GET requests for images, given a valid Imgur url.
        This method is called to extract image urls from albums/galleries of images.
        """
        img_urls = []
        imgur_hash = url.strip('/').split('/')[-1].split('?')[0]
        
        # Unpredictable API endpoint
        api_album = f'https://api.imgur.com/3/album/{imgur_hash}/images'
        api_gallery = f'https://api.imgur.com/3/gallery/album/{imgur_hash}'
        api_kwargs = {
            'method':'GET',
            'url': None,
            'headers': WallpaperScraper.imgur_headers,
            'allow_redirects': False
        }
        
        for endpoint in [api_album, api_gallery]:
            api_kwargs['url'] = endpoint
            response = requests.request(**api_kwargs)
            if response.ok:
                # Imgur is very unpredictable!
                try:
                    img_urls += [image['link'] for image in json.loads(response.text)['data']]
                except TypeError:
                    img_urls += [image['link'] for image in json.loads(response.text)['data']['images']]
                break       
        return img_urls
    
    
    @staticmethod
    def is_img_ext(url):
        """
        Determines existence of a .jpg or .png file extension from a url string
        """
        return url[-4:] in {'.jpg','.png'} 

    
    def _get_wallpaper_url(self, img_sub):
        """
        Extract wallpaper url's from reddit submissions/posts. Appends to the instance's
        'wallpaper' attribute.
        """
        if isinstance(img_sub, Submission):
            if self.is_img_ext(img_sub.url):
                self.wallpapers.append((img_sub.url, img_sub.thumbnail))
            elif 'imgur' in img_sub.url[:15]: # No need to search entire string
                self.wallpapers.extend([(img_url, None) for img_url in self.get_imgur(img_sub.url)])

In [19]:
%%timeit
WallpaperScraper('top', 'all')

10.4 s ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
test = WallpaperScraper('top', 'all')

In [27]:
len(test.wallpapers)

435

In [29]:
test.wallpapers[:10]

[('http://i.imgur.com/5yeBVeM.jpg',
  'https://b.thumbs.redditmedia.com/z68RM_D9H4pQ2BEV7QSETAU5XMZ6ruILkGeCYAH_m6A.jpg'),
 ('http://i.imgur.com/Z6kdWmA.jpg',
  'https://b.thumbs.redditmedia.com/yuNAitd2vU4Hv3YLbWeSbec663RW6IowhvoUYKrO7CU.jpg'),
 ('http://i.imgur.com/ezOgN0q.jpg',
  'https://b.thumbs.redditmedia.com/tXYU39sw5Fh34EZuaJrL6w24RFHs94xcohm1Mdx5QBc.jpg'),
 ('http://i.imgur.com/X39qMYI.jpg',
  'https://b.thumbs.redditmedia.com/x9wjjWOuxx65dsYI3U7WqPxGiTCk3ZDM2OwUSr9WhlM.jpg'),
 ('https://i.redd.it/gy65o4mk3oe01.png',
  'https://b.thumbs.redditmedia.com/KrPbXIcuyyk68qOuJbHCd9X9cuHeytNeBXqnt5H0Oxc.jpg'),
 ('https://i.imgur.com/CUGGaiZ.jpg',
  'https://b.thumbs.redditmedia.com/Kw0ZGoVkAdy_w2SWp4Al8GMivRZqyqLPW0O3BMrQIaU.jpg'),
 ('https://i.redd.it/n1ci0trfgrey.jpg',
  'https://a.thumbs.redditmedia.com/vCbBJmROViB7huvRz6foyznVN4Cpc9W1YwXWSp5z5e8.jpg'),
 ('https://i.redd.it/8kd8jy4vnem01.png',
  'https://b.thumbs.redditmedia.com/XZdg0AGYvyxs-W5iaVKSjEhB9RDDS39Rxy76329JLMk.jpg'),
 

In [30]:
it = iter(test)

In [32]:
next(it)

('http://i.imgur.com/Z6kdWmA.jpg',
 'https://b.thumbs.redditmedia.com/yuNAitd2vU4Hv3YLbWeSbec663RW6IowhvoUYKrO7CU.jpg')